In [1]:
%cd yolov7

[WinError 2] The system cannot find the file specified: 'yolov7'
c:\Users\kolak\vs code programs\Pyhton programs\yolov7


C:\Users\kolak\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [2]:
import os
print(os.getcwd())

c:\Users\kolak\vs code programs\Pyhton programs\yolov7


In [3]:
import cv2
import time
import torch
import socket
import struct
import numpy as np
from utils.general import non_max_suppression, scale_coords
from utils.torch_utils import select_device
from datetime import datetime

In [4]:
# Load pre-trained YOLOv7 model and set device
device = select_device('0' if torch.cuda.is_available() else 'cpu')
print(device)
model = torch.load('yolov7.pt', map_location=device)['model']
model.eval()

cuda:0


C:\Users\kolak\AppData\Local\Temp\ipykernel_5064\3291274807.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('yolov7.pt', map_location=device)['model']

Model(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (2): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (3): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (4): Conv(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), 

In [11]:
# Function to preprocess the frame for YOLOv7
# def preprocess_image(img, img_size=640):
#     img_resized = cv2.resize(img, (img_size, img_size))  # Resize frame
#     img_resized = img_resized[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, HWC to CHW
#     img_resized = np.ascontiguousarray(img_resized)
#     img_resized = torch.from_numpy(img_resized).float() / 255.0  # Normalize to 0-1
#     if img_resized.ndimension() == 3:
#         img_resized = img_resized.unsqueeze(0)  # Add batch dimension
#     return img_resized

def preprocess_image(img, img_size=640):
    # Get original image dimensions
    h, w = img.shape[:2]
    
    # Calculate the resize ratio while keeping aspect ratio
    scale = min(img_size / h, img_size / w)
    
    # Calculate new dimensions
    new_w, new_h = int(w * scale), int(h * scale)
    
    # Resize image
    img_resized = cv2.resize(img, (new_w, new_h))  # Resize keeping aspect ratio
    
    # Padding to fit img_size if needed (to make the image square)
    top_pad = (img_size - new_h) // 2
    bottom_pad = img_size - new_h - top_pad
    left_pad = (img_size - new_w) // 2
    right_pad = img_size - new_w - left_pad
    
    # Pad the resized image with zeros (black pixels)
    img_resized = cv2.copyMakeBorder(img_resized, top_pad, bottom_pad, left_pad, right_pad, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    
    # Convert BGR to RGB and HWC to CHW format for PyTorch
    img_resized = img_resized[:, :, ::-1].transpose(2, 0, 1)
    img_resized = np.ascontiguousarray(img_resized)
    
    # Normalize to 0-1
    img_resized = torch.from_numpy(img_resized).float() / 255.0
    
    # Add batch dimension
    if img_resized.ndimension() == 3:
        img_resized = img_resized.unsqueeze(0)
    
    return img_resized


In [12]:
# Function to detect humans in each frame
# def detect_humans(image, model, device, conf_thresh=0.25, iou_thresh=0.45, img_size=640):
#     img_preprocessed = preprocess_image(image, img_size).to(device)
#     with torch.no_grad():
#         pred = model(img_preprocessed)[0]  # Run inference

#     # Apply non-max suppression to filter out multiple detections
#     pred = non_max_suppression(pred, conf_thresh, iou_thresh, classes=[0])  # Class 0 is humans
#     return pred


# def detect_humans(image, model, device, img_size=(3840, 2160), conf_thresh=0.25, iou_thresh=0.45):
def detect_humans(image, model, device, img_size=640, conf_thresh=0.25, iou_thresh=0.45):
    # Preprocess image
    img_preprocessed = preprocess_image(image, img_size).to(device)
    
    # Check if the model is using half precision and adjust input
    if next(model.parameters()).dtype == torch.float16:
        img_preprocessed = img_preprocessed.half()  # Convert image to half precision

    with torch.no_grad():
        pred = model(img_preprocessed)[0]  # Run inference
    
    # Apply non-max suppression to filter out multiple detections
    pred = non_max_suppression(pred, conf_thresh, iou_thresh, classes=[0])  # Class 0 is humans
    
    return pred


In [13]:
# Function to plot detections on the frame
# def plot_detections(image, predictions, img_size=640):
#     img_copy = image.copy()
#     h, w = img_copy.shape[:2]
#     for det in predictions:
#         if len(det):
#             det[:, :4] = scale_coords(img_size, det[:, :4], img_copy.shape).round()
#             for *xyxy, conf, cls in det:
#                 # Draw bounding box
#                 cv2.rectangle(img_copy, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (255, 0, 0), 2)
#                 cv2.putText(img_copy, f'Person {conf:.2f}', (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

#     return img_copy

# def plot_detections(img, predictions, img_size=(3849, 2160)):
# def plot_detections(img, predictions, img_size=640):
#     img_copy = img.copy()
#     for det in predictions:
#         if len(det):
#             # Convert from normalized coordinates back to image coordinates
#             det[:, :4] = scale_coords(img_size, det[:, :4], img_copy.shape).round()
#             for *xyxy, conf, cls in det:
#                 # Draw bounding box
#                 cv2.rectangle(img_copy, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (255, 0, 0), 2)
#                 # Add confidence score
#                 cv2.putText(img_copy, f'{conf:.2f}', (int(xyxy[0]), int(xyxy[1]) - 2), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
#     return img_copy

def plot_detections(image, predictions, img_size=640):
    img_copy = image.copy()
    h, w = img_copy.shape[:2]  # Get original image dimensions

    for det in predictions:
        if len(det):
            # Convert from normalized coordinates back to original image coordinates
            det[:, :4] = scale_coords((img_size, img_size), det[:, :4], img_copy.shape).round()
            for *xyxy, conf, cls in det:
                # Draw bounding box
                cv2.rectangle(img_copy, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (255, 0, 0), 2)
                cv2.putText(img_copy, f'Person {conf:.2f}', (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    return img_copy


In [17]:
image_path=r"C:\Users\kolak\3_1_project\DataSet1\ntut_drone_train\ntut_drone_train\Drone_023\vott-csv-export\Drone_023.mp4_t-0.5.jpg"

# image_path=r"C:\Users\kolak\3_1_project\DataSet1\ntut_drone_train\ntut_drone_train\Drone_023\vott-csv-export\Drone_023.mp4_t-4.666667.jpg"
image_path=r"C:\Users\kolak\3_1_project\DataSet1\ntut_drone_train\ntut_drone_train\Drone_023\vott-csv-export\Drone_023.mp4_t-0.5.jpg"

# image_path = os.path.join(image_folder, image_file)
        
        # Load the image
frame = cv2.imread(image_path)

if frame is None:
    print(f"Error: Failed to load image.")

        # Run human detection on the image
predictions = detect_humans(frame, model, device)
        
        # Count the number of people detected
num_people = 0
for det in predictions:
    for *xyxy, conf, cls in det:
        if conf.item() >= 0.5:
            num_people += 1

        # Get the current timestamp
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # Print the detection status with the timestamp and total number of people
if num_people > 0:
    print(f"{timestamp}: {num_people} person(s) detected in {image_path}!")
            # Print bounding box details
    for det in predictions:
        if len(det):
            for *xyxy, conf, cls in det:
                if conf.item() >= 0.5:
                    x1, y1, x2, y2 = map(int, xyxy)  # Extract bounding box coordinates
                    print(f"Bounding Box: ({x1}, {y1}, {x2}, {y2}), Confidence: {conf:.2f}")
else:
    print(f"{timestamp}: No humans detected in {image_path}.")
        
        # Plot detections on the image
        # frame_with_detections = plot_detections(frame, predictions)

        # Display the image with detections
        # cv2.imshow('YOLOv7 Human Detection', frame_with_detections)
print()

    # Close all windows after processing all images
cv2.destroyAllWindows()


2024-12-24 16:34:08: 2 person(s) detected in C:\Users\kolak\3_1_project\DataSet1\ntut_drone_train\ntut_drone_train\Drone_023\vott-csv-export\Drone_023.mp4_t-0.5.jpg!
Bounding Box: (371, 298, 377, 310), Confidence: 0.70
Bounding Box: (343, 209, 350, 226), Confidence: 0.50



In [7]:
# Load image from your device

# Set the path to the folder containing images
image_folder = r"C:\Users\kolak\3_1_project\DataSet1\ntut_drone_test\ntut_drone_test\Drone_004\vott-csv-export"  # Replace with your folder path

# Get a list of all image files in the folder
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Check if any images were found
if not image_files:
    print("No images found in the specified folder.")
else:
    for image_file in image_files:
        # Construct full path to the image
        image_path = os.path.join(image_folder, image_file)
        
        # Load the image
        frame = cv2.imread(image_path)

        if frame is None:
            print(f"Error: Failed to load image {image_file}.")
            continue

        # Run human detection on the image
        predictions = detect_humans(frame, model, device)
        
        # Count the number of people detected
        num_people = 0
        for det in predictions:
            for *xyxy, conf, cls in det:
                if conf.item() >= 0.5:
                    num_people += 1

        # Get the current timestamp
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # Print the detection status with the timestamp and total number of people
        if num_people > 0:
            print(f"{timestamp}: {num_people} person(s) detected in {image_file}!")
            # Print bounding box details
            for det in predictions:
                if len(det):
                    for *xyxy, conf, cls in det:
                        if conf.item() >= 0.5:
                            x1, y1, x2, y2 = map(int, xyxy)  # Extract bounding box coordinates
                            print(f"Bounding Box: ({x1}, {y1}, {x2}, {y2}), Confidence: {conf:.2f}")
        else:
            print(f"{timestamp}: No humans detected in {image_file}.")
        
        # Plot detections on the image
        # frame_with_detections = plot_detections(frame, predictions)

        # Display the image with detections
        # cv2.imshow('YOLOv7 Human Detection', frame_with_detections)
        print()

    # Close all windows after processing all images
    cv2.destroyAllWindows()


C:\Users\kolak\AppData\Roaming\Python\Python312\site-packages\torch\functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


2024-10-24 15:21:57: 1 person(s) detected in Drone_004.mp4_t-10.166667.jpg!
Bounding Box: (375, 352, 391, 378), Confidence: 0.70

2024-10-24 15:21:58: 1 person(s) detected in Drone_004.mp4_t-10.333333.jpg!
Bounding Box: (373, 352, 389, 377), Confidence: 0.61

2024-10-24 15:21:58: 1 person(s) detected in Drone_004.mp4_t-10.5.jpg!
Bounding Box: (370, 351, 394, 401), Confidence: 0.53

2024-10-24 15:21:58: 1 person(s) detected in Drone_004.mp4_t-10.666667.jpg!
Bounding Box: (371, 352, 388, 375), Confidence: 0.64

2024-10-24 15:21:58: No humans detected in Drone_004.mp4_t-10.833333.jpg.

2024-10-24 15:21:59: 1 person(s) detected in Drone_004.mp4_t-10.jpg!
Bounding Box: (377, 351, 394, 378), Confidence: 0.69

2024-10-24 15:21:59: No humans detected in Drone_004.mp4_t-100.166667.jpg.

2024-10-24 15:21:59: No humans detected in Drone_004.mp4_t-100.333333.jpg.

2024-10-24 15:22:00: No humans detected in Drone_004.mp4_t-100.5.jpg.



KeyboardInterrupt: 

In [8]:
# import cv2

# Loop through potential device indices and test if the capture is valid
for i in range(10):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print(f"Device {i} is available.")
        cap.release()
    else:
        print(f"Device {i} is not available.")


Device 0 is available.
Device 1 is not available.
Device 2 is not available.
Device 3 is not available.
Device 4 is not available.
Device 5 is not available.
Device 6 is not available.
Device 7 is not available.
Device 8 is not available.
Device 9 is not available.


In [11]:
# Create a socket (client)
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('127.0.0.1', 65432))

In [9]:
# Capture video feed from the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open video source.")
    exit()

In [10]:

# Process the video feed frame by frame
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break
    
    # Run human detection on the current frame
    predictions = detect_humans(frame, model, device)
    
    # Check if humans are detected
    # humans_detected = any(len(det) > 0 for det in predictions)
    
    # Count the number of people detected
    # num_people = sum(len(det) for det in predictions)

    num_people=0
    for det in predictions:
        for *xyxy, conf,cls in det:
                if(conf.item()>=0.5):
                    num_people+=1

    # Get the current timestamp
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Print the detection status with the timestamp and total number of people
    # if num_people > 0:
    #     print(f"{timestamp}: {num_people} person(s) detected!")
    # else:
    #     print(f"{timestamp}: No humans detected.")

    # Print the detection status with the timestamp and number of people detected
    if num_people > 0:
        print(f"{timestamp}: {num_people} person(s) detected!")
        # Print bounding box details
        for det in predictions:
            if len(det):
                for *xyxy, conf, cls in det:
                    if conf.item()>=0.5:
                        x1, y1, x2, y2 = map(int, xyxy)  # Extract bounding box coordinates
                        print(f"Bounding Box: ({x1}, {y1}, {x2}, {y2}), Confidence: {conf:.2f}")
                        # print(type(conf.item()))
        # client_socket.sendall(b"human_detected")
    else:
        print(f"{timestamp}: No humans detected.")
        # client_socket.sendall(b"no_human")
    
    # Plot detections on the frame
    frame_with_detections = plot_detections(frame, predictions)
    
    # Display the frame with detections
    cv2.imshow('YOLOv7 Human Detection', frame_with_detections)
    
    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Exiting live feed")
        break

# Release the video capture and close windows
cap.release()
# client_socket.close()
cv2.destroyAllWindows()

C:\Users\kolak\AppData\Roaming\Python\Python312\site-packages\torch\functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


2024-12-24 12:27:19: 1 person(s) detected!
Bounding Box: (95, 121, 586, 553), Confidence: 0.96
2024-12-24 12:27:19: 1 person(s) detected!
Bounding Box: (96, 122, 590, 554), Confidence: 0.96
2024-12-24 12:27:19: 1 person(s) detected!
Bounding Box: (95, 123, 591, 554), Confidence: 0.96
2024-12-24 12:27:19: 1 person(s) detected!
Bounding Box: (96, 122, 589, 554), Confidence: 0.96
2024-12-24 12:27:19: 1 person(s) detected!
Bounding Box: (95, 123, 590, 554), Confidence: 0.96
2024-12-24 12:27:19: 1 person(s) detected!
Bounding Box: (95, 123, 589, 553), Confidence: 0.96
2024-12-24 12:27:19: 1 person(s) detected!
Bounding Box: (96, 122, 591, 553), Confidence: 0.96
2024-12-24 12:27:19: 1 person(s) detected!
Bounding Box: (96, 122, 588, 554), Confidence: 0.96
2024-12-24 12:27:19: 1 person(s) detected!
Bounding Box: (96, 123, 591, 554), Confidence: 0.96
2024-12-24 12:27:19: 1 person(s) detected!
Bounding Box: (95, 123, 592, 554), Confidence: 0.96
2024-12-24 12:27:20: 1 person(s) detected!
Boundin

In [15]:
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("Error: Could not open video source.")
    exit()

In [ ]:
# UDP video stream receiver settings
udp_ip = "0.0.0.0"  # Listen on all IP addresses (adjust IP if needed)
udp_port = 5005     # The port you're receiving video from (adjust if needed)

# Create a UDP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((udp_ip, udp_port))

# for video streaming through UDP
# Process the video feed frame by frame
while True:
    # Receive UDP packets (max size of 65535 bytes)
    data, addr = sock.recvfrom(65535)

    # Decode the received data into a frame
    frame = np.frombuffer(data, dtype=np.uint8)
    frame = cv2.imdecode(frame, cv2.IMREAD_COLOR)

    if frame is None:
        print("Error: Failed to decode image.")
        break

    # Run human detection on the current frame
    predictions = detect_humans(frame, model, device)

    # Count the number of people detected
    num_people = 0
    for det in predictions:
        for *xyxy, conf, cls in det:
            if conf.item() >= 0.5:  # Only count detections with confidence >= 0.5
                num_people += 1

    # Get the current timestamp
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Print the detection status with the timestamp and number of people detected
    if num_people > 0:
        print(f"{timestamp}: {num_people} person(s) detected!")
        # Print bounding box details
        for det in predictions:
            if len(det):
                for *xyxy, conf, cls in det:
                    if conf.item() >= 0.5:
                        x1, y1, x2, y2 = map(int, xyxy)  # Extract bounding box coordinates
                        print(f"Bounding Box: ({x1}, {y1}, {x2}, {y2}), Confidence: {conf:.2f}")
    else:
        print(f"{timestamp}: No humans detected.")

    # Plot detections on the frame
    frame_with_detections = plot_detections(frame, predictions)

    # Display the frame with detections
    cv2.imshow('YOLOv7 Human Detection', frame_with_detections)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Exiting live feed")
        break

# Release the socket and close OpenCV windows
sock.close()
cv2.destroyAllWindows()